In [5]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Načtení modelu po fine-tuningu a vstupních dat
base_model_path = "Qwen/Qwen1.5-1.8B-Chat"  # nahraď podle potřeby
csv_file = "CSV-Hashed.csv"

# Načtení base modelu
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True
)

# Načtení CSV jako text
df = pd.read_csv(csv_file)
csv_text = df.to_csv(index=False)

# Tvorba promptu pro Gwen
qwen_prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "You are a data analyst who always returns frequent itemsets in a clean format (no code)."},
        {"role": "user", "content": f"I will give you CSV and your job is to find frequent itemsets of maxlenght 5 with support 0.75. Do not use any code, it is your job to do!<csv>{csv_text}</csv>"}
    ],
    tokenize=False,
    add_generation_prompt=True
)

# Tokenizace vstupu
inputs = tokenizer(qwen_prompt, return_tensors="pt").to(model.device)

# Generování výstupu
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10000,  # Zvolena vysoká hodnota pro možnost generování rozsáhlých odpovědí
        temperature=1, # nebo 0.001, nebo 0.5
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    
# Dekódování výstupu modelu do normálního textu
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Uložení výsledku
with open("model_output.txt", "w", encoding="utf-8") as f:
    f.write(decoded_output)

print("Výstup byl uložen do 'model_output.txt'")


✅ Výstup byl uložen do 'model_output.txt'
